In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 7.9 MB/s 


In [3]:
import nltk #import library nltk
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer
from keras.models import load_model
import json
import pickle
import numpy as np
import random

model = load_model('/content/drive/MyDrive/DemoDay/chatbot_model.h5')
intents = json.loads(open('/content/drive/MyDrive/DemoDay/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/DemoDay/words.pkl', 'rb'))
classes = pickle.load(open('/content/drive/MyDrive/DemoDay/classes.pkl', 'rb'))

factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Bersihkan kalimat (tokenize dan stem) dan mengembalikan array kata
# Parameter: kalimat berbentuk string
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [5]:
# membuat bag of words dari kalimat (string) dan daftar kata yang diberikan
def bag_of_words(sentence, words, show_details = True):
    # Implementasi fungsi clean_up_sentence()
    sentence_words = clean_up_sentence(sentence)
    # Buat matriks kosakata dari N kata (akan dikembalikan sebagai bag of words)
    bag_of_words = [0] * len(words)
    for word in sentence_words:
        for i,w in enumerate(words):
            if w == word:
                # Tetapkan 1 jika kata saat ini ada di posisi kosakata, jika tidak tetapkan 0
                bag_of_words[i] = 1
                if show_details:
                    print("Found in bag: ", w)
    return(np.array(bag_of_words))

In [6]:
# Memprediksi kelas dari kalimat yang diberikan (string) menggunakan model yang diberikan
def predict_class(sentence, model):
    # Implementasi fungsi bag_of_words()
    bow = bag_of_words(sentence, words, show_details = False)
    model_result = model.predict(np.array([bow]))[0]
    # Filter prediksi di bawah ambang (0,25)
    error_threshold = 0.25
    results = [[i, result] for i, result in enumerate(model_result) if result > error_threshold]
    # Urutkan berdasarkan probabilitas
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for result in results:
        return_list.append({'intent': classes[result[0]], "probability": str(result[1])})
    return return_list

In [7]:
# Dapatkan respons acak untuk pengguna berdasarkan kelas yang diprediksi
# Parameters: kalimat masukan, daftar tag intents dan file intents json dengan tag dan respons
def get_response(sentence, intents, intents_json):
  tag = intents[0]['intent']
  intents_list = intents_json['intents']
  for intent in intents_list:
    if intent['tag'] == tag:
      response = random.choice(intent['responses'])
      break
  return response

In [8]:
# Prediksi kelas input dan pilih respons untuk dikembalikan ke pengguna
# Parameter: teks (string) yang diinputkan pengguna
def chatbot_response(text):
    # Implementasi fungsi predict_class() dan get_response()
    ints = predict_class(text, model)
    response = get_response(text, ints, intents)
    return response

In [9]:
msg = ''
print("Selamat datang di BotMojo! Ketik pesan Anda di sini atau ketik 'stop' untuk berhenti")
while(1):
    try:
      # Ambil kalimat masukan
      msg = input('User: ')
      # Cek jika kata keluar
      if msg == 'stop': break
      # kalimat Response
      response = chatbot_response(msg)
      print('BotMojo:', response)
    except KeyError:
      print("Error: Menemukan kata yang tidak dikenal.")

Selamat datang di BotMojo! Ketik pesan Anda di sini atau ketik 'stop' untuk berhenti
User: siang
BotMojo: Selamat siang, ada yang bisa dibantu?
User: kamu siapa
BotMojo: nama saya BotMojo
User: kamu bisa apa
BotMojo: Kami hanya menyediakan informasi wisata di kabupaten mojokerto
User: tujuan nya kemana saja
BotMojo: Tentukan tujuan anda, silahkan pilih dari daftar berikut: Museum trowulan, candi, maha vihara mojopahit, pendopo agung trowulan, air terjun, pemandian air panas
User: vihara
BotMojo: informasi dan lokasi maha vihara mojopahit bisa dicek di alamat berikut: http://www.mahavihara-mojopahit.or.id/
User: candi
BotMojo: hi kak, silahkan pilih daftar informasi candi berikut: brahu, bajang ratu, wringin lawang
User: wringin lawang
BotMojo: lokasi candi wringin lawang bisa dicek di alamat berikut: https://goo.gl/maps/7hK5ixAvx1NJzWPk8
User: air panas
BotMojo: hi kak, silahkan pilih daftar informasi pemandian air panas berikut: padusan atau cangar
User: padusan
BotMojo: lokasi pemand